In [1]:
"""

CycleGAN implementation adapted from Aladdin Persson:

- https://www.youtube.com/watch?v=4LktBHGCNfw
- https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Pytorch/GANs/CycleGAN

- all comments are mine; changes to the implementation are noted, along with general description of the code

"""

"""

This implementation is for a dataset of 256 x 256 patches of higher resolution images of Optical Coherence 
Tomography (OCT) scans and Gallyas Siver Stain (GSS) sample photographs

"""

'\n\nThis implementation is for a dataset of 256 x 256 patches of higher resolution images of Optical Coherence \nTomography (OCT) scans and Gallyas Siver Stain (GSS) sample photographs\n\n'

In [2]:
"""

- utils.py file, only save checkpoint is used

"""

import random, torch, os, numpy as np
import torch.nn as nn
import config
import copy

def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=config.DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
"""

config.py file

- Learning rate, lambda, epochs set to authors' original suggestion
- transforms updated to use torchvision.transforms rather than albumentations library
- file paths updated appropriately
- for testing the generated model, the RandomHorizontalFlip line is commented out

"""

import torch
import torchvision.transforms as transforms

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "CISL_Dataset/Train_Data"
VAL_DIR = "CISL_Dataset/Train_Data"
BATCH_SIZE = 1
LEARNING_RATE = 2e-4
LAMBDA_CYCLE = 10
NUM_WORKERS = 4
NUM_EPOCHS = 200
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_G_GSS = "g_gss.pth.tar"
CHECKPOINT_F_OCT = "f_oct.pth.tar"
CHECKPOINT_DISC_GSS = "disc_gss.pth.tar"
CHECKPOINT_DISC_OCT = "disc_oct.pth.tar"

transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((256, 256), antialias=True),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ],
)

In [4]:
"""

Create a class for the dataset

"""

import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class OCT_GSS_Dataset(Dataset):
    """

    - root_OCT, root_GSS: file paths to the directories storing the respective images
    -transforms: any augmentations used on the dataset; also includes conversion to tensor

    """
    def __init__(self, root_OCT, root_GSS, transform = None):
        self.root_OCT = root_OCT
        self.root_GSS = root_GSS
        self.transform = transform
        
        # create lists of files in the image directories
        self.OCT_images = os.listdir(root_OCT)
        self.GSS_images = os.listdir(root_GSS)
        
        # delete any other files from the lists of images in the directory
        for file in self.OCT_images:
            if not file.endswith(".tif"):
                self.OCT_images.remove(file)
                
        for file in self.GSS_images:
            if not file.endswith(".tif"):
                self.GSS_images.remove(file)
                
        self.length_dataset = max(len(self.OCT_images), len(self.GSS_images))
        
        self.OCT_len = len(self.OCT_images)
        self.GSS_len = len(self.GSS_images)
        
    def __len__(self):
        return self.length_dataset
    
    def __getitem__(self, index):
        """

        - read each image from the directory
        - also return the file name; this is useful to rejoin the patches into the original size images

        """
        
        # extract one file from the list of the directory, use % len() in case the amount of images in each differs
        OCT_img = self.OCT_images[index % self.OCT_len]
        GSS_img = self.GSS_images[index % self.GSS_len]
        
        OCT_path = os.path.join(self.root_OCT, OCT_img)
        GSS_path = os.path.join(self.root_GSS, GSS_img)
        
        # save the name of the file, without the directory
        OCT_file = os.path.basename(OCT_path)
        GSS_file = os.path.basename(GSS_path)
        
        OCT_img = np.array(Image.open(OCT_path).convert("RGB"))
        GSS_img = np.array(Image.open(GSS_path).convert("RGB"))
        
        # apply the transforms; this converts the numpy arrays to tensors, so transforms are necessary
        if self.transform:
            OCT_img = transforms(OCT_img)
            GSS_img = transforms(GSS_img)
            
        return OCT_img, GSS_img, OCT_file, GSS_file
        

In [5]:
"""

Create classes for the discriminators

"""

import torch
import torch.nn as nn

class Block(nn.Module):
    """

    The discriminator architecture is based on a series of Conv2d-InstanceNorm-LeakyReLU blocks,
    which are combined into a block and defined as a class

    """
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            # reflection padding is mentioned by the authors as their choice of padding
            nn.Conv2d(in_channels, out_channels, 4, stride, 1, bias = True, padding_mode = "reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace = True),
            )
        
    def forward(self, x):
        return self.conv(x)
    
class Discriminator(nn.Module):
    """

    The discriminator class is defined by instantiating a sequence of blocks

    """
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        
        # the first layer does not use Instance Normalization, so is defined without using Block
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels, features[0], kernel_size = 4, stride = 2, padding = 1, padding_mode = "reflect"),
            nn.LeakyReLU(0.2, inplace = True),
        )
        
        # the remaining convolution blocks are defined using the specified features and Block objects
        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            if feature == features[-1]:
                layers.append(Block(in_channels, feature, stride = 1))

            else:
                layers.append(Block(in_channels, feature, stride = 2))

            in_channels = feature
        
        layers.append(nn.Conv2d(in_channels, 1, kernel_size = 4, stride = 1, padding = 1, padding_mode = "reflect"))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.initial(x)
        return torch.sigmoid(self.model(x))
    

In [6]:
"""

Test the discriminator

"""

def test():
    x = torch.randn((5, 3, 256, 256))
    model = Discriminator(in_channels=3)
    preds = model(x)
    print(preds.shape)
    
if __name__ == "__main__":
    test()
    

torch.Size([5, 1, 30, 30])


In [7]:
"""

Create classes for the generators

"""

class ConvBlock(nn.Module):
    """

    The convolution blocks combine Conv2d-InstanceNorm_LeakyReLU into blocks, which are used to increase and decrease
    the number of features

    """
    def __init__(self, in_channels, out_channels, down=True, activation=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode = "reflect", **kwargs)
            
            if down
            
            else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if activation else nn.Identity(),
        )
        
    def forward(self, x):
        return self.conv(x)
    
class ResidualBlock(nn.Module):
    """

    The residual blocks combine two convolutions, without changing the size

    """
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, padding=1),
            ConvBlock(channels, channels, activation=False, kernel_size=3, padding=1),
        )
        
    def forward(self, x):
        return x + self.block(x)
    
class Generator(nn.Module):
    """

    The generator is made by instantiating the convolution and residual blocks

    """
    def __init__(self, img_channels, num_features = 64, num_residuals=9):
        super().__init__()
        # the first layer uses ReLU instead of Leaky ReLU, and is not instantiated from the ConvBlock class
        self.initial = nn.Sequential(
            nn.Conv2d(img_channels, num_features, kernel_size = 7, stride = 1, padding = 3, padding_mode = "reflect"),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace = True),
        )
        
        # two ConvBlocks are used to increase the number of features
        self.down_blocks = nn.ModuleList(
            [
                ConvBlock(num_features, num_features*2, kernel_size = 3, stride = 2, padding = 1),
                ConvBlock(num_features*2, num_features*4, kernel_size = 3, stride = 2, padding = 1),
            ]
        )
        
        # the number of residual blocks varies based on image resolution; for 256 x 256 images, 9 blocks are used
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4) for _ in range(num_residuals)]
        )
        
        # two ConvBlocks are used to decrease the number of features
        self.up_blocks = nn.ModuleList(
            [
                ConvBlock(num_features*4, num_features*2, down = False, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
                ConvBlock(num_features*2, num_features, down = False, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
            ]
        )
        
        # a final convolution layer is included to reduce to the number of image channels
        self.last = nn.Conv2d(num_features, img_channels, kernel_size = 7, stride = 1, padding = 3, padding_mode = "reflect")
        
    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.residual_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)
        return torch.tanh(self.last(x))
        

In [8]:
"""

Test the generator

"""

def test():
    img_channels = 3
    img_size = 256
    x = torch.randn((2, img_channels, img_size, img_size))
    gen = Generator(img_channels, 9)
    print(gen(x).shape)

if __name__ == "__main__":
    test()


torch.Size([2, 3, 256, 256])


In [9]:
"""

Implement a train function, and a main function to instantiate all the classes and train the network

"""

import sys
from torch.utils.data import DataLoader
from utils import save_checkpoint, load_checkpoint
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import config
from tqdm import tqdm
from torchvision.utils import save_image

"""

The train function trains the network for 1 epoch
- D_OCT & D_GSS are the discriminators for the respective image sources
- The generator G_GSS converts OCT images to GSS, and F_OCT converts GSS images to OCT
- D_optim and G_optim are the optimizers, using Adam
- L1 and MSE are the L1 and mean squared error losses
- d_scaler and g_scaler scale the step size of the gradient update between epochs

"""
def train(D_OCT, D_GSS, G_GSS, F_OCT, loader, D_optim, G_optim, L1, MSE, d_scaler, g_scaler):
    
    loop = tqdm(loader, leave=True)
    
    for idx, (OCT, GSS, OCT_file, GSS_file) in enumerate(loop):
        
        # load one image of each type
        OCT = OCT.to(config.DEVICE)
        GSS = GSS.to(config.DEVICE)
        
        with torch.cuda.amp.autocast():
            
            # generate a fake GSS image
            GSS_gen = G_GSS(OCT)
            
            # evaluate the fake and real GSS image with the discriminator
            D_GSS_true = D_GSS(GSS)
            D_GSS_gen = D_GSS(GSS_gen.detach())
            
            # calculate the loss for this discriminator
            D_GSS_true_loss = MSE(D_GSS_true, torch.ones_like(D_GSS_true))
            D_GSS_gen_loss = MSE(D_GSS_gen, torch.zeros_like(D_GSS_gen))
            
            D_GSS_loss = D_GSS_true_loss + D_GSS_gen_loss
            
            # generate a fake OCT image and evaluate with the discriminator similarly
            OCT_gen = F_OCT(GSS)
            
            D_OCT_true = D_OCT(OCT)
            D_OCT_gen = D_OCT(OCT_gen.detach())
            
            D_OCT_true_loss = MSE(D_OCT_true, torch.ones_like(D_OCT_true))
            D_OCT_gen_loss = MSE(D_OCT_gen, torch.zeros_like(D_OCT_gen))
            
            D_OCT_loss = D_OCT_true_loss + D_OCT_gen_loss
            
            # calculate the total loss of both discriminators
            D_loss = (D_GSS_loss + D_OCT_loss)/2
        
        # update the optimizer for the discriminator for each image evaluated 
        D_optim.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(D_optim)
        d_scaler.update()
        
        with torch.cuda.amp.autocast():
            
            # evaluate the loss on the fake generated images
            D_GSS_gen = D_GSS(GSS_gen)
            D_OCT_gen = D_OCT(OCT_gen)
            
            loss_G_GSS = MSE(D_GSS_gen, torch.ones_like(D_GSS_gen))
            loss_F_OCT = MSE(D_OCT_gen, torch.zeros_like(D_OCT_gen))
            
            # put the fake images through the reverse generators to compute the cycle consistency loss
            cycle_GSS = G_GSS(OCT_gen)
            cycle_OCT = F_OCT(GSS_gen)
            
            # calculate the cycle consistency loss
            cycle_loss_GSS = L1(GSS, cycle_GSS)
            cycle_loss_OCT = L1(OCT, cycle_OCT)
            
            # sum the total loss for the generators
            G_loss = (loss_G_GSS + loss_F_OCT
                     + cycle_loss_GSS * config.LAMBDA_CYCLE
                     + cycle_loss_OCT * config.LAMBDA_CYCLE
            )
        
        # update the optimizer for the generator for each image evaluated
        G_optim.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(G_optim)
        g_scaler.update()
        
        # save sample outputs
        if idx % 200 == 0:
            save_image(GSS_gen * 0.5 + 0.5, f"saved_images/GSS_{idx}.tif")
            save_image(OCT_gen * 0.5 + 0.5, f"saved_images/OCT_{idx}.tif")

           
# define a learning rate that is constant for 100 epochs, and linearly decays to 0 for the next 100, as described
# by the original authors
def lr_linear_decay(epoch):
    if epoch < 100:
        return config.LEARNING_RATE
    else:
        return config.LEARNING_RATE*(1 - ((epoch - 100) / 100.0))         



def main():
    
    # define an instance of both discriminators and generators
    D_OCT = Discriminator(in_channels = 3).to(config.DEVICE)
    D_GSS = Discriminator(in_channels = 3).to(config.DEVICE)
    G_GSS = Generator(img_channels = 3, num_residuals = 9).to(config.DEVICE)
    F_OCT = Generator(img_channels = 3, num_residuals = 9).to(config.DEVICE)
    
    # instantiate the optimizers, using betas as described by the original authors
    D_optim = optim.Adam(
        list(D_OCT.parameters()) + list(D_GSS.parameters()),
        lr = config.LEARNING_RATE,
        betas = (0.5, 0.999),
    )
    
    G_optim = optim.Adam(
        list(G_GSS.parameters()) + list(F_OCT.parameters()),
        lr = config.LEARNING_RATE,
        betas = (0.5, 0.999),
    )
    
    # define the learning rate schedulers using the above learning rate decay
    D_scheduler = lr_scheduler.LambdaLR(D_optim, lr_linear_decay)
    
    G_scheduler = lr_scheduler.LambdaLR(G_optim, lr_linear_decay)
    
    # call the L1 and MSE loss functions to use in training
    L1 = nn.L1Loss()
    MSE = nn.MSELoss()
    
    # create an option to load a partially trained model (not used)
    if config.LOAD_MODEL:
        load_checkpoint(
            config.CHECKPOINT_G_GSS,
            G_GSS,
            G_optim,
            config.LEARNING_RATE,
        )
        load_checkpoint(
            config.CHECKPOINT_F_OCT,
            F_OCT,
            F_optim,
            config.LEARNING_RATE,
        )
        load_checkpoint(
            config.CHECKPOINT_DISC_GSS,
            D_GSS,
            D_optim,
            config.LEARNING_RATE,
        )
        load_checkpoint(
            config.CHECKPOINT_DISC_OCT,
            D_OCT,
            D_optim,
            config.LEARNING_RATE,
        )

    # define an instance of the dataset for training
    dataset = OCT_GSS_Dataset(
        root_OCT = config.TRAIN_DIR + "/OCT_Patch",
        root_GSS = config.TRAIN_DIR + "/GSS_Patch",
        transform = config.transforms,
    )
    
    # define the data loader
    loader = DataLoader(
        dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        num_workers=config.NUM_WORKERS,
        pin_memory=True,
    )
    
    # define the gradient scalers
    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()

    for epoch in range(config.NUM_EPOCHS):
        
        print("Epoch: ", epoch)
        
        train(D_OCT, D_GSS, G_GSS, F_OCT, loader, D_optim, G_optim, L1, MSE, d_scaler, g_scaler)

        # update the learning rate as necessary after each epoch
        D_scheduler.step()
        G_scheduler.step()
        
        # save and overwrite the model after each epoch
        if config.SAVE_MODEL:
            save_checkpoint(G_GSS, G_optim, filename=config.CHECKPOINT_G_GSS)
            save_checkpoint(F_OCT, G_optim, filename=config.CHECKPOINT_F_OCT)
            save_checkpoint(D_GSS, D_optim, filename=config.CHECKPOINT_DISC_GSS)
            save_checkpoint(D_OCT, D_optim, filename=config.CHECKPOINT_DISC_OCT)

if __name__ == "__main__":
    main()


  0%|          | 0/677 [00:00<?, ?it/s]

Epoch:  0


100%|██████████| 677/677 [00:55<00:00, 12.20it/s]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


  0%|          | 0/677 [00:00<?, ?it/s]

...

=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


  0%|          | 0/677 [00:00<?, ?it/s]

Epoch:  199


100%|██████████| 677/677 [00:50<00:00, 13.50it/s]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


In [13]:
"""

Implement a test function and main function to generate new GSS samples using the trained network
It is possible to generate OCT samples as well, but practically there is no use for them, so they are excluded

"""

def test(G_GSS, loader):
    
    loop = tqdm(loader, leave=True)
    
    # for the loaded OCT images, generate GSS samples and save each image
    for idx, (OCT, GSS, OCT_file, GSS_file) in enumerate(loop):
        
        OCT = OCT.to(config.DEVICE)
        GSS = GSS.to(config.DEVICE)
        
        with torch.cuda.amp.autocast():
            
            GSS_gen = G_GSS(OCT)
            
            # The OCT file has a coordinate corresponding to the patch of the original image it comes from
            # this is used in the saved file name to reconstruct the full image
            save_image(GSS_gen * 0.5 + 0.5, f"CISL_Dataset/saved_final_images/GSSfrom_" + OCT_file[0])

def main():
    
    # instantiate a dataset object using images prepared for testing
    test_dataset = OCT_GSS_Dataset(
        root_OCT = config.TRAIN_DIR + "/Test_OCT_Patch",
        root_GSS = config.TRAIN_DIR + "/Test_GSS_Patch",
        transform = config.transforms,
    )
    
    # make a data loader with the test dataset
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        pin_memory=True,
    )
    
    # load the pretrained model, only for the generator that outputs GSS images
    GSS_trained = Generator(img_channels = 3, num_residuals = 9).to(config.DEVICE)
    checkpoint = torch.load('g_gss.pth.tar')
    GSS_trained.load_state_dict(checkpoint['state_dict'])
    
    # run the test function
    test(GSS_trained, test_loader)
    
if __name__ == "__main__":
    main()

100%|██████████| 88/88 [00:03<00:00, 25.19it/s]
